# 📘 LangChain Study Reference Notebook
This notebook provides a structured reference for working with **LangChain** (with Ollama as backend LLM).
Each section includes:
- Concept explanation
- Example code
- Notes / references


## 1. Setup
Before working with LangChain, ensure you have the required packages installed.
We’ll also define a small helper function to interact with Ollama locally.


In [2]:
import os
import requests

def get_completion_local(prompt, model="llama3"):
    """Send a prompt to a local Ollama server and return the response."""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()['response']

In [3]:
chat = get_completion_local(
"""Translate the text below into American English
in a calm and respectful tone.

Text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls!```""",
model="llama3"
)
print(chat)

Here's the translation:

"Wow, I'm really upset that my blender lid came loose and covered my kitchen walls in a mess!"

I've kept the tone calm and respectful as per your request. Let me know if you need any further assistance!


## 2. LangChain with Ollama
LangChain integrates with Ollama via the `ChatOllama` class. This allows you to invoke models hosted locally through Ollama.


In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3", temperature=0.1)
llm.invoke("Hello, how are you?").content

"I'm just a language model, I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! It's great to chat with you. How can I help you today?"

## 3. Prompts
LangChain provides `ChatPromptTemplate` to structure prompts. You can define placeholders and fill them dynamically.


In [5]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_template("Translate this into French: {text}")
prompt = template.format_messages(text="Good morning!")
print(prompt)

[HumanMessage(content='Translate this into French: Good morning!', additional_kwargs={}, response_metadata={})]


## 4. Output Parsing
Use `StructuredOutputParser` to enforce structured responses from the model. This ensures reliable parsing of outputs into JSON-like structures.


In [6]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="answer", description="The direct answer"),
    ResponseSchema(name="confidence", description="Confidence score between 0 and 1")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(
    "Answer the question: {question}\n{format_instructions}"
).partial(format_instructions=format_instructions)

messages = prompt.format_messages(question="What is LangChain?")
raw_output = llm.invoke(messages)
structured = parser.parse(raw_output.content)

print("Raw:", raw_output.content)
print("Parsed:", structured)

Raw: ```json
{
  "answer": "LangChain is a large language model that uses a combination of transformer-based architectures and chain-like structures to generate coherent and context-aware text.",
  "confidence": "0.9"
}
```

Note: The confidence score is an estimate based on my training data, and it may not reflect the actual accuracy or reliability of the answer.
Parsed: {'answer': 'LangChain is a large language model that uses a combination of transformer-based architectures and chain-like structures to generate coherent and context-aware text.', 'confidence': '0.9'}


## 5. Memory
LangChain 0.3.x uses **message history** and `RunnableWithMessageHistory`.
- `MessagesPlaceholder("history")` defines where past messages appear in the prompt.
- `InMemoryChatMessageHistory` stores previous exchanges.
- `RunnableWithMessageHistory` automatically injects and updates conversation history.


In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

model = ChatOllama(model="llama3")
parser = StrOutputParser()
base_chain = prompt | model | parser

store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chat = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

config = {"configurable": {"session_id": "demo"}}

print(chat.invoke({"input": "Hi, please remember my favorite color is blue."}, config=config))
print(chat.invoke({"input": "What is my favorite color?"}, config=config))

I'd be happy to! I've taken note that your favorite color is indeed blue. If you need any reminders or have questions, feel free to ask!
Easy one! Your favorite color is... BLUE!


## 6. Adding Messages Manually
You can inject messages into history directly:
- `add_user_message("...")`
- `add_ai_message("...")`

### Advantages:
- Preload user profile or facts
- Restore state across sessions
- Simulate interactions in testing


In [8]:
history = get_session_history("demo")
history.add_user_message("Remind me about LangChain.")
history.add_ai_message("LangChain is a framework for building with LLMs.")

for m in history.messages:
    print(type(m).__name__, ":", m.content)

HumanMessage : Hi, please remember my favorite color is blue.
AIMessage : I'd be happy to! I've taken note that your favorite color is indeed blue. If you need any reminders or have questions, feel free to ask!
HumanMessage : What is my favorite color?
AIMessage : Easy one! Your favorite color is... BLUE!
HumanMessage : Remind me about LangChain.
AIMessage : LangChain is a framework for building with LLMs.


## 7. References
- [LangChain Documentation](https://python.langchain.com/)
- [Ollama Documentation](https://ollama.com/)
- [LangChain Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)
- [Memory in LangChain](https://python.langchain.com/docs/expression_language/how_to/message_history/)
